# DAT210x - Programming with Python for DS

## Module6- Lab3

This code is intentionally missing! Read the directions on the course lab page!

In [3]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import svm
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsClassifier

In [4]:
X = pd.read_csv('Datasets/parkinsons.data')
y = X['status']
X.drop(['name', 'status'], axis=1, inplace=True)

In [14]:
X.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.02971,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.04368,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.03590,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.03772,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.04465,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [16]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: status, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=7)

In [27]:
model = svm.SVC()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.81355932203389836

That accuracy was just too low to be useful. We need to get it up. One way you could go about doing that would be to manually try a bunch of combinations of C, and gamma values for your rbf kernel. But that could literally take forever. Also, you might unknowingly skip a pair of values that would have resulted in a very good accuracy.

Instead, lets get the computer to do what computers do best. Program a naive, best-parameter search by creating nested for-loops. The outer for-loop should iterate a variable C from  to , using  unit increments. The inner for-loop should increment a variable gamma from to , using  unit increments. As you know, Python ranges won't allow for float intervals, so you'll have to do some research on NumPy ARanges, if you don't already know how to use them.

Since the goal is to find the parameters that result in the model having the best accuracy score, you'll need a best_score = 0 variable that you initialize outside of the for-loops. Inside the inner for-loop, create an SVC model and pass in the C and gamma parameters its class constructor. Train and score the model appropriately. If the current best_score is less than the model's score, update the best_score being sure to print it out, along with the C and gamma values that resulted in it.

After running your assignment again, what is the highest accuracy score you are able to get?

In [40]:
c_range = np.arange(0.05, 2, 0.05)
gamma_range = np.arange(0.001, 0.1, 0.001)

In [41]:
best_c = 0
best_gamma = 0
best_score = 0

for c in c_range:
    for gamma in gamma_range:
        model = svm.SVC(C=c, gamma=gamma, kernel='rbf')
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        if score > best_score:
            best_c = c
            best_gamma = gamma
            best_score = score

In [42]:
print(best_c)
print(best_gamma)
print(best_score)


1.65
0.005
0.915254237288


Look at the units on those columns: Hz, %, Abs, dB, etc. What happened to transforming your data? With all of those units interacting with one another, some pre-processing is surely in order.

Right after you preform the train/test split but before you train your model, inject SciKit-Learn's pre-processing code. Unless you have a good idea which one is going to work best, you're going to have to try the various pre-processors one at a time, checking to see if they improve your predictive accuracy.

Experiment with Normalizer(), MaxAbsScaler(), MinMaxScaler(), KernelCenterer(), and StandardScaler().

After trying all of these scalers, what is the new highest accuracy score you're able to achieve?

In [6]:
scalers = [preprocessing.Normalizer(),
           preprocessing.MaxAbsScaler(),
           preprocessing.MinMaxScaler(),
           preprocessing.KernelCenterer(),
           preprocessing.StandardScaler()]
c_range = np.arange(0.05, 2, 0.05)
gamma_range = np.arange(0.001, 0.1, 0.001)

In [56]:
best_c = 0
best_gamma = 0
best_score = 0

for scaler in scalers:
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=7)
    
    # Normalize
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Loop over c and gamma
    for c in c_range:
        for gamma in gamma_range:
            model = svm.SVC(C=c, gamma=gamma, kernel='rbf')
            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)
            if score > best_score:
                best_c = c
                best_gamma = gamma
                best_score = score
                best_scaler = scaler

In [57]:
print(best_c)
print(best_gamma)
print(best_score)
print(best_scaler)

1.55
0.097
0.932203389831
StandardScaler(copy=True, with_mean=True, with_std=True)


The accuracy score keeps creeping upwards. Let's have one more go at it. Remember how in a previous lab we discovered that SVM's are a bit sensitive to outliers and that just throwing all of our unfiltered, dirty or noisy data at it, particularly in high-dimensionality space, can actually cause the accuracy score to suffer?

Well, let's try to get rid of some useless features. Immediately after you do the pre-processing, run PCA on your dataset. The original dataset has 22 columns and 1 label column. So try experimenting with PCA n_component values between 4 and 14. Are you able to get a better accuracy?

If you are not, then forget about PCA entirely. However if you are able to get a higher score, then be *sure* keep that accuracy score in mind, and comment out all the PCA code for now.

In the same spot, run Isomap on the data. Manually experiment with every inclusive combination of n_neighbors between 2 and 5, and n_components between 4 and 6. Are you able to get a better accuracy?

If you are not, then forget about isomap entirely. However if you are able to get a higher score, then be *sure* keep that figure in mind.

If either PCA or Isomap helped you out, then uncomment out the appropriate transformation code so that you have the highest accuracy possible.

What is your highest accuracy score on this assignment to date?

In [18]:
pca_components = range(4,14,1)

best_c = 0
best_gamma = 0
best_score = 0
best_pca_components = 0


for scaler in scalers:
    for n_components in pca_components:
        
        X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=7)

        # Normalize
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    
        # Reduce dimensionality
        pca_model = PCA(n_components=n_components, random_state=7)
        pca_model.fit(X_train)
        X_train = pca_model.transform(X_train)
        X_test = pca_model.transform(X_test)

        # Loop over c and gamma
        for c in c_range:
            for gamma in gamma_range:
                model = svm.SVC(C=c, gamma=gamma, kernel='rbf')
                model.fit(X_train, y_train)
                score = model.score(X_test, y_test)
                if score > best_score:
                    best_c = c
                    best_gamma = gamma
                    best_score = score
                    best_scaler = scaler
                    best_pca_components = n_components

In [19]:
print(best_c)
print(best_gamma)
print(best_score)
print(best_scaler)
print(best_pca_components)

1.65
0.098
0.932203389831
StandardScaler(copy=True, with_mean=True, with_std=True)
7


In [20]:
isomap_components = range(4,14,1)
isomap_neighbors = range(4,6,1)

best_c = 0
best_gamma = 0
best_score = 0
best_pca_components = 0


for scaler in scalers:
    
    # Reduce dimensionality
    for n_components in isomap_components:
        
        for n_neighbors in isomap_neighbors:
            
            X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=7)
    
            # Normalize
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)

            isomap_model = Isomap(n_neighbors=n_neighbors, n_components = n_components)

            isomap_model.fit(X_train)
            X_train = isomap_model.transform(X_train)
            X_test = isomap_model.transform(X_test)

            # Loop over c and gamma
            for c in c_range:
                for gamma in gamma_range:
                    model = svm.SVC(C=c, gamma=gamma, kernel='rbf')
                    model.fit(X_train, y_train)
                    score = model.score(X_test, y_test)
                    if score > best_score:
                        best_c = c
                        best_gamma = gamma
                        best_score = score
                        best_scaler = scaler
                        best_isomap_components = n_components
                        best_isomap_neighbors = n_neighbors

In [21]:
print(best_c)
print(best_gamma)
print(best_score)
print(best_scaler)
print(best_isomap_components)
print(best_isomap_neighbors)

1.15
0.098
0.949152542373
StandardScaler(copy=True, with_mean=True, with_std=True)
4
4
